In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
model = tf.keras.models.load_model('LSTM.h5',compile = True)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 10)           200000    
                                                                 
 bidirectional (Bidirectiona  (None, 300)              193200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 40)                12040     
                                                                 
 dense_1 (Dense)             (None, 3)                 123       
                                                                 
Total params: 405,363
Trainable params: 405,363
Non-trainable params: 0
_________________________________________________________________


In [4]:
data_predict = pd.read_excel("Book1.xlsx")
data_predict = data_predict.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
print(f"total null {data_predict.isnull().sum()}")
print(f"total null {data_predict.isna().sum()}")

total null text    0
dtype: int64
total null text    0
dtype: int64


In [5]:
data_predict['text'] = data_predict['text'].replace({'"':'',
                                 '\d+':'',
                                 ':':'',
                                 ';':'',
                                 '#':'',
                                 '_':'',
                                 ',': '',
                                 "'": '',
                                  }, regex=True)

data_predict['text'] = data_predict['text'].str.replace(r'[https]+[?://]+[^\s<>"]+|www\.[^\s<>"]+[?()]+[(??)]+[)*]+[(\xa0]+[-&gt]', "",regex=True)

data_predict['text'] = data_predict['text'].replace('\n','', regex=True)

data_predict['text'] = data_predict['text'].replace({'\.':'','(/)':'','\(':'','\)':''},regex=True)
data_predict['text'] = data_predict['text'].replace('[\.:"]','',regex =True)
data_predict['text'] = data_predict['text'].replace('\?', '', regex=True)
data_predict['text'] = data_predict['text'].replace('\!', '', regex=True)
data_predict['text'] = data_predict['text'].replace('\*', '', regex=True)
data_predict['text'] = data_predict['text'].replace('\%', '', regex=True)
data_predict['text'] = data_predict['text'].replace('\&', '', regex=True)
data_predict['text'] = data_predict['text'].replace('\~', '', regex=True)
data_predict['text'] = data_predict['text'].replace('\=', ' ', regex=True)
data_predict['text'] = data_predict['text'].replace('\-', '', regex=True)
data_predict['text'] = data_predict['text'].replace('@', ' ', regex=True)
data_predict['text'] = data_predict['text'].str.replace(r'[^\w\s]+', '')
#data_predict['text'].astype(str)
data_predict['text'] = data_predict['text'].replace('\s+', ' ', regex=True)
data_predict['text'] = data_predict['text'].str.strip()
data_predict['text'] = data_predict['text'].str.lower()


C:\Users\W I N D O W S\AppData\Local\Temp\ipykernel_19716\3460411168.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  data_predict['text'] = data_predict['text'].str.replace(r'[^\w\s]+', '')


In [6]:
data = data_predict['text'].tolist()
data[:5]

['kalian tidak akan ada hubungan yang sihat jika berpasangan dengan si toksik fikirannya negatif dan dia akan perlahanlahan memakan kamu kalau jumpa begini cepatcepat angkat kaki',
 'eunchayang kalo si garam salah emang sangat pantas dihukum tapi kalo dia gak salah ya jangan jahatin dia lagi bilang mental health tai kucing tapi bilang wajar garam dihujat gara gara agensi gak gerak cepat dan garam masih muncul awas aja yang ngehujat dituntut hybe gara gara ini',
 'semua sama dimata hukumteorifaktanyakasus yg menyenggol umat mayoritas selalu lbh cepat reaksinyasi bangsat sukro sampai hari ini msh bebas kontol',
 'semua sama dimata hukumteorifaktanyakasus yg menyenggol umat mayoritas selalu lbh cepat reaksinyasi bangsat sukro sampai hari ini msh bebas kontol',
 'kalian tidak akan ada hubungan yang sihat jika berpasangan dengan si toksik fikirannya negatif dan dia akan perlahanlahan memakan kamu kalau jumpa begini cepatcepat angkat kaki']

In [7]:
vocab_size = 20000
embedding_dim = 10
max_length = 150
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

In [8]:
X = data
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(data)
padded = pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
predic = model.predict(padded)


42/42 [==============================] - 6s 81ms/step

In [9]:
y_label = {0:"negative",1:"neutral",2:"positive"}
sentimen = []
for x in predic:
    sentimen.append(y_label[np.argmax(x)])


In [10]:
data_clean = pd.DataFrame({'tweet':data_predict['text'],'sentimen':sentimen})
data_clean.to_excel("sentiment.xlsx")
#'count':data_predict['count']

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
